### IMPORT LIBRARY AND OPEN SPARK SESSION

In [ ]:
import findspark
findspark.init("C:\spark")
import pandas as pd

In [84]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [30]:
import pyspark
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession, functions as F
spark = SparkSession.builder\
                    .master("yarn")\
                    .appName("Clustering_KNN")\
                    .getOrCreate()
sc = spark.sparkContext

### READ DATA

In [67]:
df = spark.read.csv("flo100k_data.csv", header=True, inferSchema=True, sep="|")

### EXPLORE DATA

In [68]:
df.limit(5).toPandas()

C:\spark\python\pyspark\sql\pandas\conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
C:\spark\python\pyspark\sql\pandas\conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
C:\spark\python\pyspark\sql\pandas\conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
C:\spark\python\pyspark\sql\pandas\conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,_c0,master_id,order_channel,platform_type,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,last_order_date_new
0,0,b3ace094-a17f-11e9-a2fc-000d3a38a36f,Offline,Offline,Offline,2019-02-23 12:59:17,2019-02-23 12:59:17,NaT,2019-02-23 12:59:17,NaN,1.0,212.98,0.00,None,2019-02-23
1,1,c57d7c4c-a950-11e9-a2fc-000d3a38a36f,Offline,OmniChannel,Offline,2019-12-01 16:48:09,2019-12-01 16:48:09,NaT,2019-12-01 16:48:09,NaN,1.0,199.98,0.00,None,2019-12-01
2,2,602897a6-cdac-11ea-b31f-000d3a38a36f,Offline,Offline,Offline,2020-07-24 15:49:47,2020-07-24 15:49:47,NaT,2020-07-24 15:49:47,NaN,1.0,140.49,0.00,[ERKEK],2020-07-24
3,3,388e4c4e-af86-11e9-a2fc-000d3a38a36f,Mobile,Online,Mobile,2018-12-31 07:22:07,2018-12-31 07:22:07,2018-12-31 07:22:07,NaT,1.0,NaN,0.00,174.99,None,2018-12-31
4,4,80664354-adf0-11eb-8f64-000d3a299ebf,Desktop,Online,Desktop,2021-05-05 21:07:02,2021-05-05 22:39:36,2021-05-05 22:39:36,NaT,2.0,NaN,0.00,283.95,[],2021-05-05


In [69]:
df_count = df.count()
print(df_count)

100000


In [15]:
len(df.columns)

15

In [70]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- master_id: string (nullable = true)
 |-- order_channel: string (nullable = true)
 |-- platform_type: string (nullable = true)
 |-- last_order_channel: string (nullable = true)
 |-- first_order_date: timestamp (nullable = true)
 |-- last_order_date: timestamp (nullable = true)
 |-- last_order_date_online: timestamp (nullable = true)
 |-- last_order_date_offline: timestamp (nullable = true)
 |-- order_num_total_ever_online: double (nullable = true)
 |-- order_num_total_ever_offline: double (nullable = true)
 |-- customer_value_total_ever_offline: double (nullable = true)
 |-- customer_value_total_ever_online: double (nullable = true)
 |-- interested_in_categories_12: string (nullable = true)
 |-- last_order_date_new: date (nullable = true)



In [71]:
df = df.drop("_c0")

### NULL CHECK

In [47]:
def null_count(dataframe, col_name):
    nc = dataframe.select(col_name).filter(
                                              (F.col(col_name) == "NA") |
                                              (F.col(col_name).isNull()) |
                                              (F.col(col_name) == "")).count()
    return nc

In [34]:
df.select("interested_in_categories_12").filter(
                                              (F.col("interested_in_categories_12") == "NA") |
                                              (F.col("interested_in_categories_12").isNull()) |
                                              (F.col("interested_in_categories_12") == "")).count()

56590

In [43]:
null_count(df, "interested_in_categories_12")

56590

In [83]:
for col_name in df.dtypes:
    nc = null_count(df, col_name[0])
    if nc > 0:
        print("{} has {} null value and % {}".format(col_name[0], nc, (nc/df_count)*100))

last_order_date_online has 70784 null value and % 70.784
last_order_date_offline has 21703 null value and % 21.703
interested_in_categories_12 has 56590 null value and % 56.589999999999996


### IS MASTER ID UNIQUE?

In [73]:
df.select("master_id").distinct().count() == df.count()

True

### DATA UNDERSTANDING

In [74]:
df.groupBy(["platform_type", "order_channel"]).count().show()

+-------------+-------------+-----+
|platform_type|order_channel|count|
+-------------+-------------+-----+
|       Online|       Mobile| 6451|
|       Online|  Android App| 8728|
|  OmniChannel|      Offline| 4793|
|      Offline|      Offline|65991|
|  OmniChannel|  Android App| 3261|
|       Online|      Ios App| 3008|
|  OmniChannel|      Ios App|  956|
|       Online|      Desktop| 3253|
|  OmniChannel|       Mobile| 2061|
|  OmniChannel|      Desktop| 1498|
+-------------+-------------+-----+



### CREATE NEW COLUMN FROM DF COLUMNS

In [75]:
df = df.fillna(0, subset=["order_num_total_ever_online","order_num_total_ever_offline"])

In [99]:
df1 = df.withColumn("order_num_total", F.col("order_num_total_ever_online") + F.col("order_num_total_ever_offline"))\
        .withColumn("customer_value_total", F.col("customer_value_total_ever_online") + F.col("customer_value_total_ever_offline"))


In [100]:
df1.limit(3).toPandas()

,master_id,order_channel,platform_type,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,last_order_date_new,order_num_total,customer_value_total
0,b3ace094-a17f-11e9-a2fc-000d3a38a36f,Offline,Offline,Offline,2019-02-23 12:59:17,2019-02-23 12:59:17,NaT,2019-02-23 12:59:17,0.0,1.0,212.98,0.0,None,2019-02-23,1.0,212.98
1,c57d7c4c-a950-11e9-a2fc-000d3a38a36f,Offline,OmniChannel,Offline,2019-12-01 16:48:09,2019-12-01 16:48:09,NaT,2019-12-01 16:48:09,0.0,1.0,199.98,0.0,None,2019-12-01,1.0,199.98
2,602897a6-cdac-11ea-b31f-000d3a38a36f,Offline,Offline,Offline,2020-07-24 15:49:47,2020-07-24 15:49:47,NaT,2020-07-24 15:49:47,0.0,1.0,140.49,0.0,[ERKEK],2020-07-24,1.0,140.49


### ANALYSIS ORDER CHANNEL AND PLATFORM

In [89]:
df1.groupBy("order_channel").agg({"master_id": "count"}).show()

+-------------+----------------+
|order_channel|count(master_id)|
+-------------+----------------+
|  Android App|           11989|
|       Mobile|            8512|
|      Ios App|            3964|
|      Desktop|            4751|
|      Offline|           70784|
+-------------+----------------+



In [90]:
df1.groupBy("order_channel").agg({"order_num_total": "mean"}).show()

+-------------+------------------+
|order_channel|   avg(_num_total)|
+-------------+------------------+
|  Android App|  3.50971724080407|
|       Mobile| 2.798637218045113|
|      Ios App| 3.377648839556004|
|      Desktop| 2.538623447695222|
|      Offline|1.6003475361663653|
+-------------+------------------+



In [91]:
df1.groupBy("order_channel").agg({"customer_value_total": "sum"}).show()

+-------------+-------------------------+
|order_channel|sum(customer_value_total)|
+-------------+-------------------------+
|  Android App|         6388294.10000052|
|       Mobile|       3334506.8500001733|
|      Ios App|        2253391.419999976|
|      Desktop|       1790716.2999999432|
|      Offline|     1.5442516930013401E7|
+-------------+-------------------------+



In [189]:
df1.groupBy("order_channel").agg(F.count("master_id"), F.mean("order_num_total"), F.sum("customer_value_total")).show()

+-------------+----------------+--------------------+-------------------------+
|order_channel|count(master_id)|avg(order_num_total)|sum(customer_value_total)|
+-------------+----------------+--------------------+-------------------------+
|  Android App|           11989|    3.50971724080407|         6388294.10000052|
|       Mobile|            8512|   2.798637218045113|       3334506.8500001733|
|      Ios App|            3964|   3.377648839556004|        2253391.419999976|
|      Desktop|            4751|   2.538623447695222|       1790716.2999999432|
|      Offline|           70784|  1.6003475361663653|     1.5442516930013401E7|
+-------------+----------------+--------------------+-------------------------+



In [92]:
df1.groupBy("platform_type").agg({"master_id": "count"}).show()

+-------------+----------------+
|platform_type|count(master_id)|
+-------------+----------------+
|  OmniChannel|           12569|
|       Online|           21440|
|      Offline|           65991|
+-------------+----------------+



In [93]:
df1.groupBy("platform_type").agg({"order_num_total": "mean"}).show()

+-------------+------------------+
|platform_type|   avg(_num_total)|
+-------------+------------------+
|  OmniChannel|3.4947092051873656|
|       Online|2.6207089552238805|
|      Offline|1.5837917291751906|
+-------------+------------------+



In [94]:
df1.groupBy("platform_type").agg({"customer_value_total": "sum"}).show()

+-------------+-------------------------+
|platform_type|sum(customer_value_total)|
+-------------+-------------------------+
|  OmniChannel|       6294476.6600005785|
|       Online|        8678690.260001639|
|      Offline|     1.4236258680012075E7|
+-------------+-------------------------+



In [190]:
df1.groupBy("platform_type").agg(F.count("master_id"), F.mean("order_num_total"), F.sum("customer_value_total")).show()

+-------------+----------------+--------------------+-------------------------+
|platform_type|count(master_id)|avg(order_num_total)|sum(customer_value_total)|
+-------------+----------------+--------------------+-------------------------+
|  OmniChannel|           12569|  3.4947092051873656|       6294476.6600005785|
|       Online|           21440|  2.6207089552238805|        8678690.260001639|
|      Offline|           65991|  1.5837917291751906|     1.4236258680012075E7|
+-------------+----------------+--------------------+-------------------------+



### CALCULATION OF K-MEANS METRICS 

In [102]:
columns = ["master_id", "last_order_date_new", "order_num_total", "customer_value_total"]
rfm = df1.select(columns)

In [103]:
rfm.limit(5).toPandas()

,master_id,last_order_date_new,order_num_total,customer_value_total
0,b3ace094-a17f-11e9-a2fc-000d3a38a36f,2019-02-23,1.0,212.98
1,c57d7c4c-a950-11e9-a2fc-000d3a38a36f,2019-12-01,1.0,199.98
2,602897a6-cdac-11ea-b31f-000d3a38a36f,2020-07-24,1.0,140.49
3,388e4c4e-af86-11e9-a2fc-000d3a38a36f,2018-12-31,1.0,174.99
4,80664354-adf0-11eb-8f64-000d3a299ebf,2021-05-05,2.0,283.95


In [130]:
last_date_order = rfm.agg({"last_order_date_new": "max"}).collect()[0][0]

In [193]:
# last_date = rfm.orderBy("last_order_date_new", ascending = False).first()["last_order_date_new"]
# print(last_date)

2021-05-30


In [131]:
print(last_date_order)

2021-05-30


In [197]:
rfm1 = rfm.withColumn("Recency", F.expr(f"datediff('{last_date_order}', last_order_date_new) + 2"))

In [195]:
rfm1.printSchema()

root
 |-- master_id: string (nullable = true)
 |-- last_order_date_new: date (nullable = true)
 |-- order_num_total: double (nullable = false)
 |-- customer_value_total: double (nullable = true)
 |-- Recency: integer (nullable = true)



In [196]:
rfm1.limit(3).toPandas()

,master_id,last_order_date_new,order_num_total,customer_value_total,Recency
0,b3ace094-a17f-11e9-a2fc-000d3a38a36f,2019-02-23,1.0,212.98,829
1,c57d7c4c-a950-11e9-a2fc-000d3a38a36f,2019-12-01,1.0,199.98,548
2,602897a6-cdac-11ea-b31f-000d3a38a36f,2020-07-24,1.0,140.49,312


In [137]:
rfm1 = rfm1.withColumnRenamed("order_num_total", "Frequency").withColumnRenamed("customer_value_total", "Monetary")

In [141]:
rfm2 = rfm1.drop("last_order_date_new")

In [143]:
rfm2.limit(5).toPandas()

,master_id,Frequency,Monetary,Recency
0,b3ace094-a17f-11e9-a2fc-000d3a38a36f,1.0,212.98,829
1,c57d7c4c-a950-11e9-a2fc-000d3a38a36f,1.0,199.98,548
2,602897a6-cdac-11ea-b31f-000d3a38a36f,1.0,140.49,312
3,388e4c4e-af86-11e9-a2fc-000d3a38a36f,1.0,174.99,883
4,80664354-adf0-11eb-8f64-000d3a299ebf,2.0,283.95,27


### VECTOR ASSEMBLER

In [149]:
assembler_cols = rfm2.columns[1:]

In [144]:
from pyspark.ml.feature import VectorAssembler

In [158]:
assembler = VectorAssembler()\
    .setHandleInvalid("skip")\
    .setInputCols(assembler_cols)\
    .setOutputCol("unscaled_features")

### FEATURE SCALER

In [159]:
from pyspark.ml.feature import StandardScaler

In [162]:
scaler = StandardScaler()\
    .setInputCol("unscaled_features")\
    .setOutputCol("features")

### PIPELINE

In [164]:
from pyspark.ml import Pipeline

In [165]:
pipeline_obj = Pipeline().setStages([assembler, scaler])

In [166]:
pipeline_model = pipeline_obj.fit(rfm2)

In [167]:
pipeline_df = pipeline_model.transform(rfm2)

In [168]:
pipeline_df.limit(5).toPandas()

,master_id,Frequency,Monetary,Recency,unscaled_features,features
0,b3ace094-a17f-11e9-a2fc-000d3a38a36f,1.0,212.98,829,"[1.0, 212.98, 829.0]","[0.3277012075046895, 0.4598810576445184, 3.247..."
1,c57d7c4c-a950-11e9-a2fc-000d3a38a36f,1.0,199.98,548,"[1.0, 199.98, 548.0]","[0.3277012075046895, 0.43181056393910594, 2.14..."
2,602897a6-cdac-11ea-b31f-000d3a38a36f,1.0,140.49,312,"[1.0, 140.49, 312.0]","[0.3277012075046895, 0.3033556662056456, 1.222..."
3,388e4c4e-af86-11e9-a2fc-000d3a38a36f,1.0,174.99,883,"[1.0, 174.99, 883.0]","[0.3277012075046895, 0.37785043796231704, 3.45..."
4,80664354-adf0-11eb-8f64-000d3a299ebf,2.0,283.95,27,"[2.0, 283.95, 27.0]","[0.655402415009379, 0.6131243605886045, 0.1057..."


### K-MEANS MODEL AND OPTIMUM CLUSTER DETERMINATION

In [169]:
from pyspark.ml.clustering import KMeans

In [170]:
def compute_kmeans_model(df, k):
    kmeansObject = KMeans() \
        .setSeed(142) \
        .setK(k)
    
    return kmeansObject.fit(df)

In [171]:
from pyspark.ml.evaluation import ClusteringEvaluator
evaluator = ClusteringEvaluator()

In [172]:
for k in range(2,11):
    kmeans_model = compute_kmeans_model(pipeline_df, k)
    
    transformed_df = kmeans_model.transform(pipeline_df)
    
    score = evaluator.evaluate(transformed_df)
    
    print("k: {}, score: {}" .format(k, score))

k: 2, score: 0.999540331314793
k: 3, score: 0.6549749959656543
k: 4, score: 0.6752012569444599
k: 5, score: 0.5023405880556393
k: 6, score: 0.5425218220645989
k: 7, score: 0.5532151404232838
k: 8, score: 0.6500417695963127
k: 9, score: 0.547918043476468
k: 10, score: 0.5083959076286391


In [173]:
kmeans_model = compute_kmeans_model(pipeline_df, 4)

### PREDICTION

In [174]:
transformed_df = kmeans_model.transform(pipeline_df)

In [175]:
transformed_df.limit(10).toPandas()

,master_id,Frequency,Monetary,Recency,unscaled_features,features,prediction
0,b3ace094-a17f-11e9-a2fc-000d3a38a36f,1.0,212.98,829,"[1.0, 212.98, 829.0]","[0.3277012075046895, 0.4598810576445184, 3.247...",0
1,c57d7c4c-a950-11e9-a2fc-000d3a38a36f,1.0,199.98,548,"[1.0, 199.98, 548.0]","[0.3277012075046895, 0.43181056393910594, 2.14...",0
2,602897a6-cdac-11ea-b31f-000d3a38a36f,1.0,140.49,312,"[1.0, 140.49, 312.0]","[0.3277012075046895, 0.3033556662056456, 1.222...",0
3,388e4c4e-af86-11e9-a2fc-000d3a38a36f,1.0,174.99,883,"[1.0, 174.99, 883.0]","[0.3277012075046895, 0.37785043796231704, 3.45...",0
4,80664354-adf0-11eb-8f64-000d3a299ebf,2.0,283.95,27,"[2.0, 283.95, 27.0]","[0.655402415009379, 0.6131243605886045, 0.1057...",0
5,47511f36-aeb4-11e9-a2fc-000d3a38a36f,1.0,139.98,933,"[1.0, 139.98, 933.0]","[0.3277012075046895, 0.30225443914489475, 3.65...",0
6,77f7c318-3407-11eb-9a15-000d3a38a36f,1.0,269.90,182,"[1.0, 269.9, 182.0]","[0.3277012075046895, 0.5827866346992934, 0.713...",0
7,399d6dd2-ecf1-11ea-9369-000d3a38a36f,1.0,95.73,272,"[1.0, 95.73, 272.0]","[0.3277012075046895, 0.206706797109164, 1.0656...",0
8,b3d4a6f2-a368-11e9-a2fc-000d3a38a36f,3.0,207.94,250,"[3.0, 207.94, 250.0]","[0.9831036225140686, 0.44899834316180465, 0.97...",0
9,42fdccd4-f1d1-11ea-bde9-000d3a38a36f,1.0,134.95,226,"[1.0, 134.95, 226.0]","[0.3277012075046895, 0.2913933173496467, 0.885...",0


### DESCRIPTIVE STATISTICS

In [185]:
transformed_df.groupBy("prediction").agg(F.mean("Frequency"), F.stddev_samp("Frequency"), F.max("Frequency"), F.min("Frequency")).show()

+----------+------------------+----------------------+--------------+--------------+
|prediction|    avg(Frequency)|stddev_samp(Frequency)|max(Frequency)|min(Frequency)|
+----------+------------------+----------------------+--------------+--------------+
|         1|226.83333333333334|    104.16221323813481|         377.0|          89.0|
|         0| 2.032801968118087|    2.3953926084982684|         126.0|           1.0|
+----------+------------------+----------------------+--------------+--------------+



In [187]:
transformed_df.groupBy("prediction").agg(F.mean("Monetary"), F.stddev_samp("Monetary"), F.max("Monetary"), F.min("Monetary")).show()

+----------+------------------+---------------------+-------------+-------------+
|prediction|     avg(Monetary)|stddev_samp(Monetary)|max(Monetary)|min(Monetary)|
+----------+------------------+---------------------+-------------+-------------+
|         1| 31003.60666666667|   12517.356352959945|     51178.23|     17005.69|
|         0|290.25145468715596|    387.3742833124113|     18905.75|          0.0|
+----------+------------------+---------------------+-------------+-------------+



In [188]:
transformed_df.groupBy("prediction").agg(F.mean("Recency"), F.stddev_samp("Recency"), F.max("Recency"), F.min("Recency")).show()

+----------+------------------+--------------------+------------+------------+
|prediction|      avg(Recency)|stddev_samp(Recency)|max(Recency)|min(Recency)|
+----------+------------------+--------------------+------------+------------+
|         1|             343.5|  262.99866919815395|         657|          24|
|         0|413.60598635918154|   255.2391576237783|        1097|           2|
+----------+------------------+--------------------+------------+------------+

